In [ ]:
# Function

In [ ]:
from pathlib import Path
import optuna
import subprocess
import pickle
import numpy as np
import pandas as pd
import time

In [ ]:
def random_choose_index():
    global idx, df
    return np.random.choice(idx, p=df.prob)

In [ ]:
def run_command(command):
    result = subprocess.run(command, capture_output=True)
    assert (
        result.returncode == 0
    ), f"""
command:
{' '.join(command)}
returncode: {result.returncode}
stderr:
{result.stderr.decode()}
stdout:
{result.stdout.decode()}
"""
    return result

In [ ]:
def run_a_instance(mean_arr, std_arr, rho_arr, indv_ants_arr):
    global debug_mode, debug_time, acopp_dir, df, sol_dir, postfix

    df_idx = random_choose_index()
    instance_name = df.loc[df_idx].instance
    acopp_profit = df.loc[df_idx].acopp_profit

    command = [
        'python3',
        f'{acopp_dir}/run.py',
        '--acopp_dir',
        str(acopp_dir),
        '--instance_name',
        instance_name,
        '--run_only',
        '--experiment',
        # '--no_log',
        '--sol_dir',
        str(sol_dir),
        '--silent',
        '1',
        "--postfix",
        str(postfix),
        
        '--no_default',
        "--chain_flags",
        f"--adapt_evap --cmaes --mean_ary {mean_arr} --std_ary {std_arr} --adpt_rho {rho_arr} --indv_ants {indv_ants_arr}",
    ]

    if debug_mode:
        command += ["--time", str(debug_time)]
    
    result = run_command(command)
    stdout_log = result.stdout.decode()
    profit = int(stdout_log)
    
    gain_percent = (profit - acopp_profit) / acopp_profit * 100
    return gain_percent

In [ ]:
def to_arr_flag(a_list):
    arr_flag = map(str, a_list)
    arr_flag = ":".join(arr_flag)
    return arr_flag

In [ ]:
def objective(trial):
    global n_run_each_trail

    indv_ants = trial.suggest_int("indv_ants", 2, 50)
    min_indv_ants = trial.suggest_int("min_indv_ants", 2, indv_ants)
    max_indv_ants = trial.suggest_int("max_indv_ants", indv_ants, 50)

    rho = trial.suggest_float("rho", 0.01, 0.99)
    min_rho = trial.suggest_float("min_rho", 0.01, rho)
    max_rho = trial.suggest_float("max_rho", rho, 0.99)

    alpha_mean = trial.suggest_float("alpha_mean", 0.01, 10)
    # mean + k*std <= upper_bound       <=> std <= (upper_bound - mean) / k
    # mean - k*std >= lower_bound       <=> std <= (mean - lower_bound) / k
    std_upper_bound = min((10 - alpha_mean), (alpha_mean - 0.01)) / 2
    std_upper_bound = max(std_upper_bound, 0.01)
    alpha_std = trial.suggest_float("alpha_std", 0.01, std_upper_bound)

    beta_mean = trial.suggest_float("beta_mean", 0.01, 10)
    std_upper_bound = min((10 - beta_mean), (beta_mean - 0.01)) / 2
    std_upper_bound = max(std_upper_bound, 0.01)
    beta_std = trial.suggest_float("beta_std", 0.01, std_upper_bound)

    par_a_mean = trial.suggest_float("par_a_mean", 0.01, 1)
    std_upper_bound = min((1 - par_a_mean), (par_a_mean - 0.01)) / 2
    std_upper_bound = max(std_upper_bound, 0.01)
    par_a_std = trial.suggest_float("par_a_std", 0.01, std_upper_bound)

    par_b_mean = trial.suggest_float("par_b_mean", 0.01, 1)
    std_upper_bound = min((1 - par_b_mean), (par_b_mean - 0.01)) / 2
    std_upper_bound = max(std_upper_bound, 0.01)
    par_b_std = trial.suggest_float("par_b_std", 0.01, std_upper_bound)

    par_c_mean = trial.suggest_float("par_c_mean", 0.01, 1)
    std_upper_bound = min((1 - par_c_mean), (par_c_mean - 0.01)) / 2
    std_upper_bound = max(std_upper_bound, 0.01)
    par_c_std = trial.suggest_float("par_c_std", 0.01, std_upper_bound)

    mean_arr = to_arr_flag([alpha_mean, beta_mean, par_a_mean, par_b_mean, par_c_mean])
    std_arr = to_arr_flag([alpha_std, beta_std, par_a_std, par_b_std, par_c_std])
    rho_arr = to_arr_flag([rho, min_rho, max_rho])
    indv_ants_arr = to_arr_flag([indv_ants, min_indv_ants, max_indv_ants])

    objective_value = 0
    for i in range(n_run_each_trail):
        objective_value += run_a_instance(mean_arr, std_arr, rho_arr, indv_ants_arr) / n_run_each_trail

    return objective_value

# load dataframe

In [ ]:
df = pd.read_csv("./es_ant_gain_percent.csv")
df.info()

In [ ]:
df.describe()

In [ ]:
idx = np.arange(len(df.instance))

In [ ]:
haha = np.random.choice(idx, p=df.prob)
haha, df.loc[haha].instance

# Config

In [ ]:
acopp_dir = Path("../")
save_path = Path("./study.pkl")
sol_dir = Path("./solutions")
postfix = str(time.time())

total_trial = 1000
n_jobs = 3
n_run_each_trail = 3
save_each_n_trial = 1

sampler = optuna.samplers.TPESampler()
idx = np.arange(len(df.instance))

debug_mode = False
# debug_mode = True
debug_time = 10
if debug_mode:
    total_trial = 2
    n_jobs = 2
    save_each_n_trial = 1

# Build

In [ ]:
command = [
    'python3',
    f'{acopp_dir}/run.py',
    '--acopp_dir',
    str(acopp_dir),
    '--build_only',
    '--experiment'
    ]
result = run_command(command)
print(result.stdout.decode())

# Run

In [ ]:
# study = optuna.create_study(direction='maximize', sampler=sampler)
# with open(save_path, "wb") as f:
#     pickle.dump(study, f)

In [ ]:
with open(save_path, "rb") as f:
    study = pickle.load(f)
    
while (len(study.trials) < total_trial):
    study.optimize(objective, n_trials=save_each_n_trial, n_jobs=n_jobs)
    with open(save_path, "wb") as f:
        pickle.dump(study, f)